In [9]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb

In [10]:
import torch
torch.cuda.empty_cache()

In [11]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./cascade_rcnn/cascade_rcnn_swin-t-p4-w7_fpn_1x_coco.py')

root='/opt/ml/detection/dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.runner.max_epochs = 100
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [13]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.24s)
creating index...
index created!


[
 CocoDataset Train dataset with number of images 4883, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
 | 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [16]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-03-25 15:02:53,098 - mmdet - INFO - Start running, host: root@6f46492e33de, work_dir: /opt/ml/CV_18_ysw/Mmdetection/_boost_/work_dirs/cascade_rcnn_r50_fpn_1x_trash
2022-03-25 15:02:53,100 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                

learning_rate,▁▁▂▂▃▃▄▄▅▅▆▇▇███████████████████████████
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▁▂▁▁▂▁▁▁▂▁▂▁▁▁▁▂▁▁▁▁▁▁▁
train/loss,█▂▁▂▁▁▁▁▁▁▂▁▂▂▁▂▂▂▂▁▂▂▁▁▁▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁
train/loss_rpn_bbox,▇▄▅██▆▅▆▄▂▆▄▇▅▄▆▅▄▅▄▃▃▃▃▄▅▃▆▄▃▃▄▃▅▂▁▃▃▂▂
train/loss_rpn_cls,█▅▃▂▂▂▂▂▁▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s0.acc,▁███▇▇██▇█▇█▇▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇▇▇▇█▇███
train/s0.loss_bbox,▁▄▅███▇▆▆▅▇▆█▇▅▆▆▅▆▅▆▆▅▅▅▆▅▆▆▅▅▆▄▅▄▃▅▄▄▃
train/s0.loss_cls,█▁▁▂▂▂▁▁▁▁▂▁▂▂▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/s1.acc,▁█████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
train/s1.loss_bbox,▁▂▂▃▃▄▅▅▆▆▇▇██▇▇█▇▆▇██▇▇▇▇▇▇█▇▇█▇▇▇▆▇▇▆▆


2022-03-25 15:03:33,987 - mmdet - INFO - Epoch [1][50/1221]	lr: 4.995e-06, eta: 20:26:56, time: 0.603, data_time: 0.055, memory: 5597, loss_rpn_cls: 0.6334, loss_rpn_bbox: 0.0449, s0.loss_cls: 1.3152, s0.acc: 70.4922, s0.loss_bbox: 0.0664, s1.loss_cls: 0.8177, s1.acc: 52.3877, s1.loss_bbox: 0.0198, s2.loss_cls: 0.4182, s2.acc: 54.2822, s2.loss_bbox: 0.0027, loss: 3.3183, grad_norm: 29.6266
2022-03-25 15:04:09,340 - mmdet - INFO - Epoch [1][100/1221]	lr: 9.990e-06, eta: 22:12:02, time: 0.707, data_time: 0.014, memory: 5597, loss_rpn_cls: 0.4347, loss_rpn_bbox: 0.0418, s0.loss_cls: 0.3172, s0.acc: 94.2500, s0.loss_bbox: 0.1344, s1.loss_cls: 0.0924, s1.acc: 97.3965, s1.loss_bbox: 0.0407, s2.loss_cls: 0.0310, s2.acc: 98.6006, s2.loss_bbox: 0.0057, loss: 1.0978, grad_norm: 4.5796
2022-03-25 15:04:47,893 - mmdet - INFO - Epoch [1][150/1221]	lr: 1.499e-05, eta: 23:30:02, time: 0.771, data_time: 0.016, memory: 5597, loss_rpn_cls: 0.2188, loss_rpn_bbox: 0.0451, s0.loss_cls: 0.3124, s0.acc: 93.6

learning_rate,████▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,█▇█▇▄▁▂▃▃▄▂▃▃▂▃▂▃▃▃▃▂▃▃▃▂▃▂▃▂▃▄▃▃▄▃▃▃▃▃▃
train/loss,█▇▆▅▃▂▂▂▂▃▂▁▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▂▁▂▂▂▂▁▁▁▂▁▁▂
train/loss_rpn_bbox,█▇▇▅▄▂▃▃▃▄▃▁▂▂▂▂▂▃▃▃▂▃▃▂▂▂▂▂▁▃▂▃▃▁▁▃▃▁▃▂
train/loss_rpn_cls,█▇▅▄▃▁▂▂▃▃▂▁▂▂▁▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▂▂▂▁▂▁▁▂▁
train/s0.acc,▁▂▂▄▆▇▇▇▇▅▇█▇▇▇▇▇▇▇▇█▇▇▇▇▇█▇█▇▇▇▇███▇█▇▇
train/s0.loss_bbox,█▆▆▄▃▂▂▂▂▄▂▁▂▂▂▂▂▃▃▂▁▂▂▂▂▂▂▂▁▃▂▂▂▁▂▁▂▁▂▂
train/s0.loss_cls,█▇▆▅▃▂▂▂▂▃▂▁▂▁▂▂▂▂▂▂▁▂▂▂▂▁▁▂▁▂▂▂▁▁▁▁▁▁▁▂
train/s1.acc,▁▂▃▄▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇█▇▇▇█▇▇
train/s1.loss_bbox,█▆▇▅▄▁▂▃▂▄▂▁▂▂▂▂▃▃▃▂▁▂▂▂▂▂▁▂▁▃▂▂▂▁▂▁▂▁▁▂


In [17]:
import torch
torch.cuda.empty_cache()